In [1]:
from imports import *
from FeatureExtraction import *
from VSM import *

In [2]:
_PreProcessor = PreProcessor()
UC_documents, code_documents, UCTokens, CodeTokens = _PreProcessor.setup(
            "./UC", "./CC"
        )
UCTokens.update(CodeTokens)

In [4]:
featureExtraction = FeatureExtraction(UCTokens)

###### used for all ######
tfidf_matrix_uc, tfidf_matrix_code = featureExtraction.TFIDFVectorizer(UC_documents, code_documents)
UC_count_matrix, code_count_matrix = featureExtraction.CountVectorizerModel(UC_documents, code_documents, 'train')
# the values of the count matrices are normalized

#-------------------------14 IR based features--------------------------#

# 1) Vector space model
cosine_similarities_feature = featureExtraction.VectorSpaceModel(tfidf_matrix_uc, tfidf_matrix_code)
print('cosine similarity', cosine_similarities_feature.shape)  ## UC * Documents

# 2) Latent semantic analysis
LSA_similraities_feature = featureExtraction.LSA(tfidf_matrix_uc, tfidf_matrix_code, 'train')
print('LSA similarity', LSA_similraities_feature.shape)

# 3) Latent Dirichlet Allocation


# 4) Jensen-Shannon(JS)
JS_features = featureExtraction.JensenShannon(UC_count_matrix, code_count_matrix)

# 5)  Okapi BM25

# 6) Language Model with Dirichlet


#------------------------pre-retrieval (21 metrics)--------------------------#

#------------------------post-retrieval (7 metrics)--------------------------#

#------------------------Document Statistics Features--------------------------#
num_terms_code, num_terms_UC, num_unique_terms_code, num_unique_terms_UC, num_overlapping_terms = featureExtraction.DocumentStatistics(UC_documents, code_documents)



cosine similarity (58, 116)
LSA similarity (58, 116)


In [ ]:
# num_components = min(tfidf_matrix_uc.shape[0], tfidf_matrix_code.shape[1])
# num_components = min(num_components, 100)
# LSA_model = TruncatedSVD(n_components=num_components)

# # if train_or_test == "train":
# LSA_data_useCases = LSA_model.fit_transform(tfidf_matrix_uc)
# LSA_data_codes = LSA_model.fit_transform(tfidf_matrix_code) 
# # else:
# #     LSA_data_useCases = LSA_model.transform(tfidf_matrix_uc)
# #     LSA_data_codes = LSA_model.transform(tfidf_matrix_code) 

# LSA_similraity_matrix = cosine_similarity(LSA_data_useCases, LSA_data_codes)
# print(LSA_similraity_matrix)

In [ ]:
# test = FeatureExtraction()
# test.LSA(tfidf_matrix_uc, tfidf_matrix_code, "train")

In [ ]:
#usecases (100*usecases_nu)
#code (100*codes_nu)
#LSA_similraity_matrix (usecases_nu*codes_nu)
#d1,d2,d3,d4
#u1,u2,u3,u4
#u1*d1,u1*d2,u1*d3 ...
#d1*u1,d1*u2,d1*u3 ...
#each row coresspond between the use cases and each document
# [[d1*c1 d1*c2],[d2*c1 d2*c2]]

# LSA_data_codes=np.array([[1,2,3,4,5] , [6,7,8,9,10]])
# LSA_data_useCases=np.array([[11,12,13,14,15] , [16,17,18,19,21],[2,5,9,11,42]])

# LSA_data_codes_norm=np.repeat(np.linalg.norm(LSA_data_codes,axis=1),LSA_data_useCases.shape[0])
# LSA_data_useCases_norm=np.repeat(np.linalg.norm(LSA_data_useCases,axis=1),LSA_data_codes.shape[0])

# LSA_data_codes_norm=LSA_data_codes_norm.reshape(LSA_data_codes.shape[0],LSA_data_useCases.shape[0]).T 
# LSA_data_useCases_norm=LSA_data_useCases_norm.reshape(LSA_data_useCases.shape[0],LSA_data_codes.shape[0])

# print(LSA_data_codes_norm)
# print(LSA_data_useCases_norm)

# LSA_similraity_matrix=cosine_similarity(LSA_data_useCases, LSA_data_codes)

# print(LSA_similraity_matrix)

## Jensen-Shannon

In [ ]:
test = FeatureExtraction()
test.JensenShannon(UCTokens, UC_documents, code_documents)

In [ ]:
def getOverlap(query_term_list: list, total_query_list: list) -> int:
    query_term_list = pd.Series(query_term_list)
    query_term_list = query_term_list.nlargest(10)
    query_term_set = set(query_term_list.index.values)

    total_query_list = pd.Series(total_query_list)
    total_query_list = total_query_list.nlargest(10)
    total_query_set = set(total_query_list.index.values)

    return(len(query_term_set & total_query_set))



#running the query as a whole
feature_extraction=FeatureExtraction(UCTokens)
UC_count_matrix, code_count_matrix = feature_extraction.CountVectorizerModel(UC_documents, code_documents, 'train')

### Run the original query q, and obtain the result list R 
total_score_whole_query_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, code_count_matrix) 

### Run each individual query term qt in the original query as a separate query and obtain the result list Rt.
overall_queries_score = list()
for code_idx, code in enumerate(code_documents):
    code_words = code.split(" ")
    query__term_overlap_list = list()
    for word in code_words:
        if word == '':
            continue
        # word_count_matrix = np.zeros((code_count_matrix.shape[0], code_count_matrix.shape[1]))
        word_index = feature_extraction.code_vocab_index[word]
        # word_count_matrix[:, word_index] = code_count_matrix[:, word_index]
        # print(feature_extraction.count_vectorizer.vocabulary_["if"])
        # print(code_count_matrix[:, 752])
        # print(code_count_matrix[:, word_index])
        term_vector=np.zeros((1, UC_count_matrix.shape[1]))
        term_vector[:,word_index]=code_count_matrix[code_idx, word_index]
        query_term_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, term_vector)
        # print(query_term_jensen_shannon.shape)
        query_term_len = query_term_jensen_shannon.shape[0] * query_term_jensen_shannon.shape[1]
        query__term_overlap_list.append(getOverlap(query_term_jensen_shannon.reshape(query_term_len), total_score_whole_query_jensen_shannon[:, code_idx]))

    overall_queries_score.append(stdev(query__term_overlap_list))
# print(overall_queries_score)
        # print(total_score_query_term_jensen_shannon)

        
    # break




# print(total_score_whole_query_jensen_shannon[:,7])
# print(total_score_whole_query_jensen_shannon[:,6])
# print(code[7])
# for i, temp in enumerate(code[7]):
#     if temp != 0:
#         print(i)
# # 3 features to be called
#1st case => if the query is the code
# total_score_whole_query_jensen_shannon=feature_extraction.JensenShannon(UC_documents,,"train") #58*116

In [ ]:
# def getOverlap(query_term_list: np.ndarray, total_query_set: set) -> int:
#     query_term_set = set(np.argpartition(query_term_list, -10)[-10:])
#     return len(query_term_set & total_query_set)

# def SubqueryOverlapTerms(query_term: str, code_idx: int, UC_count_matrix: np.ndarray, code_count_matrix: np.ndarray, feature_extraction: object) -> np.ndarray:
    
#     word_index = feature_extraction.code_vocab_index[query_term]
#     term_vector = np.zeros((1, UC_count_matrix.shape[1])) #1*
#     term_vector[:,word_index]=code_count_matrix[code_idx, word_index]

#     query_term_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, term_vector)
#     query_term_len = query_term_jensen_shannon.shape[0] * query_term_jensen_shannon.shape[1]

#     query_term_jensen_shannon = query_term_jensen_shannon.reshape(query_term_len)
#     return query_term_jensen_shannon

# def SubqueryOverlapCode(code_idx, code, UC_count_matrix: np.ndarray, code_count_matrix: np.ndarray, feature_extraction: object, total_query_list: np.ndarray):

#     code_idx = int(code_idx)
#     code_words = code.split(" ")
#     code_words = np.ascontiguousarray(code_words[0:-1])
#     vSubqueryOverlap = np.vectorize(lambda query_term: SubqueryOverlapTerms(query_term, code_idx, UC_count_matrix, code_count_matrix, feature_extraction), otypes=[np.ndarray])
    
#     query_term_jensen_shannon = vSubqueryOverlap(code_words)
#     total_query_set = set(np.argpartition(total_query_list[:, code_idx], -10)[-10:])
   
#     vGetOverlap = np.vectorize(lambda query_term_jensen_shannon: getOverlap(query_term_jensen_shannon, total_query_set), otypes=[np.int16])
#     query_term_overlap = vGetOverlap(query_term_jensen_shannon)
    
#     overall_queries_score = np.std(query_term_overlap)
#     return overall_queries_score

In [ ]:
#feature_extraction.JensenShannon
# def SubqueryOverlap(feature_extraction:object, query:list, feature_extraction_method:Callable, document_count_matrix:np.ndarray, query_count_matrix:np.ndarray) -> np.ndarray:
    
#     ### Run the original query q, and obtain the result list R 
#     total_score_query_feature_extraction = feature_extraction_method(document_count_matrix, query_count_matrix)

#     query_tuples =  np.ascontiguousarray(list(enumerate(query)))
    
#     ### Run each individual query term qt in the original query as a separate query and obtain the result list Rt.
#     vSubqueryOverlapCode = np.vectorize(lambda code_idx, code_doc: SubqueryOverlapCode(code_idx, code_doc, document_count_matrix, query_count_matrix, feature_extraction, total_score_query_feature_extraction), otypes=[np.float16])

#     overall_queries_score = vSubqueryOverlapCode(query_tuples[:,0], query_tuples[:,1])
#     return overall_queries_score

## Subquery overlap test

In [ ]:
#client = Client(processes=False, threads_per_worker=4,n_workers=2, memory_limit='2GB')

feature_extraction = FeatureExtraction(UCTokens)
UC_count_matrix, code_count_matrix = feature_extraction.CountVectorizerModel(UC_documents, code_documents, 'train')

 
code_queries_score_JensenShannon = feature_extraction.SubqueryOverlap(feature_extraction, code_documents, feature_extraction.JensenShannon, UC_count_matrix, code_count_matrix)
UC_queries_score_JensenShannon = feature_extraction.SubqueryOverlap(feature_extraction, UC_documents, feature_extraction.JensenShannon, code_count_matrix, UC_count_matrix)

# tfidf_matrix_uc,tfidf_matrix_code=feature_extraction.TFIDFVectorizer(UC_documents,code_documents)
# # code_queries_score_vector_space = feature_extraction.SubqueryOverlap(feature_extraction, code_documents,cosine_similarity, tfidf_matrix_uc, #tfidf_matrix_code)
# uc_queries_score_vector_space = feature_extraction.SubqueryOverlap(feature_extraction, UC_documents,cosine_similarity, tfidf_matrix_code,tfidf_matrix_uc)

# # print(code_queries_score_vector_space.shape)
# print(uc_queries_score_vector_space.shape)
# print(uc_queries_score_vector_space)


In [ ]:
feature_extraction=FeatureExtraction(UCTokens)
UC_count_matrix, code_count_matrix = feature_extraction.CountVectorizerModel(UC_documents, code_documents, 'train')

### Run the original query q, and obtain the result list R 
total_score_whole_query_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, code_count_matrix) 
query_term_set = np.argsort(total_score_whole_query_jensen_shannon,axis=0)[-50:,:]
new_ranked_documents=[] #50 documents

print(query_term_set.shape)
# print(total_score_whole_query_jensen_shannon[0])

## Play ground yasmenya ama nshof eh el araf da

In [ ]:
from imports import *
from PreProcessor import *
import os
import numpy as np

CodeTokens = set()
UCTokens = set()
UC_documents = list()
code_documents = list()
TotalTokens=set()
entropy_uc = []
entropy_code = []
variance_uc = {}
variance_code = {}
SCQ_uc={}
SCQ_code={}
term_co_occurrences_uc = {}
PMI_uc = {}
term_co_occurrences_code= {}
PMI_code = {}

for filename in os.listdir("./UC"):
    filepath = os.path.join("./UC", filename)
    tokens = UCPreProcessor(filepath)
    UC_documents.append(tokens)
    UCTokens.update(tokens.split())


for filename in os.listdir("./CC"):
    filepath = os.path.join("./CC", filename)
    tokens = CodePreProcessor(filepath)
    code_documents.append(tokens)
    CodeTokens.update(tokens.split())



In [ ]:
vectorizer_uc = TfidfVectorizer(vocabulary=UCTokens)
tf_matrix_uc=vectorizer_uc.fit_transform(UC_documents)
idf_uc = vectorizer_uc.idf_



# Get the vocabulary from the TfidfVectorizer object
vocabulary_uc = vectorizer_uc.get_feature_names_out()

# Pair terms with their corresponding document frequencies
term_document_frequencies = dict(zip(vocabulary_uc,idf_uc))

print(term_document_frequencies)


In [ ]:
def get_overlap(query_term_list: np.ndarray, total_query_list: np.ndarray) -> int:
    query_term_set = set(np.argpartition(-query_term_list, 10)[:10])
    total_query_set = set(np.argpartition(-total_query_list, 10)[:10])

    return len(query_term_set & total_query_set)

temp1 = np.array([90, 100, 200, 30, 400, 500, 600, 700, 800, 900, 100])
temp2 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

np.argsort(-temp1)[:10]

# temp1 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# temp2 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# indices_temp1 = np.argpartition(-temp1, 11)[:10]
# sorted_indices_temp1 = indices_temp1[np.argsort(temp1[indices_temp1])]

# print(sorted_indices_temp1)


### Document Statistics Features

In [ ]:
num_terms_code =[]
for document in code_documents:
    num_terms_code.append(len(document.split()))

num_terms_code=np.array(num_terms_code)
print(num_terms_code)

num_terms_UC =[]
for document in UC_documents:
    num_terms_UC.append(len(document.split()))

num_terms_UC=np.array(num_terms_UC)
print(num_terms_UC)

num_unique_terms_code =[]
for document in code_documents:
    num_unique_terms_code.append(len(set(document.split())))

num_unique_terms_code=np.array(num_unique_terms_code)
print(num_unique_terms_code)

num_unique_terms_UC =[]
for document in UC_documents:
    num_unique_terms_UC.append(len(set(document.split())))

num_unique_terms_UC=np.array(num_unique_terms_UC)
print(num_unique_terms_UC)

# each column represent D1 intersect with all usecases, then column2= D2 intersect with all usescases....
#if want to access the intesection between a specific usecase and all coressponding documents you will access the row index coressponding to this usecase 
num_overlapping_terms_code =np.zeros((len(UC_documents),len(code_documents)))
for uc_index,uc in enumerate(UC_documents):
    for code_index,code in enumerate(code_documents):
        num_overlapping_terms_code[uc_index][code_index] = 100*(len(set(uc.split()).intersection(set(code.split()))))/(num_unique_terms_UC[uc_index]+num_unique_terms_code[code_index])

print(num_overlapping_terms_code)


